# Basic Template Starter

In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
from tqdm.notebook import tqdm

import time
from urllib.parse import urljoin

In [5]:
%pip install -r requirements.txt
%pip install jupyter_contrib_nbextensions
!jupyter contrib nbextension install --user
!jupyter nbextension enable --py widgetsnbextension
!jupyter labextension install @jupyter-widgets/jupyterlab-manager


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Traceback (most recent call last):
  File "c:\Users\bendh\anaconda3\envs\test_2\Lib\site-packages\jupyter_contrib_core\notebook_compat\nbextensions.py", line 6, in <module>
    from notebook.extensions import BaseExtensionApp
ModuleNotFoundError: No module named 'notebook.extensions'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\bendh\anaconda3\envs\test_2\Lib\site-packages\jupyter_contrib_core\notebook_compat\nbextensions.py", line 10, in <module>
    from notebook.nbextensions import BaseNBExtensionApp
ModuleNotFoundError: No module named 'notebook.nbextensions'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\bendh\anaconda3\envs\test_2\Scripts\jupyter-contrib.EXE\__main__.py", line 7, in <module>
  File "c:\Users\bendh\anaconda3\env

In [6]:
## Get the data!

# Toggle whether to use the whole data or just a sample
USE_SAMPLE = True

# Scrape the given URL for all parquet file links
def get_parquets(url):
    response = requests.get(url)
    if response.status_code != 200:
        return []
    
    soup = BeautifulSoup(response.text, "html.parser")
    return [
        a["href"].strip()
        for a in soup.find_all("a", href=True)
        if a["href"].strip().endswith(".parquet")
        and (not USE_SAMPLE or ("yellow" in a["href"] and "2024" in a["href"]))
    ]

# Download a single file while displaying a progress bar
def download_file(url, save_path, position=1):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))

    with open(save_path, 'wb') as file, tqdm(
        desc=os.path.basename(save_path),
        total=total_size,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
        position=position,
        leave=False,
        dynamic_ncols=True,
    ) as bar:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                file.write(chunk)
                bar.update(len(chunk))

# Download and process all parquet files found on the webpage
def download_and_read_parquet_files(url, save_dir="data"):
    os.makedirs(save_dir, exist_ok=True)
    
    links = get_parquets(url)
    dataframes = []
    
    with tqdm(total=len(links), desc="Processing files", unit="file", position=0, leave=True) as file_bar:
        for link in links:
            full_link = urljoin(url, link)
            filename = os.path.join(save_dir, os.path.basename(link))

            if not os.path.exists(filename):
                download_file(full_link, filename, position=1)
                time.sleep(5)

            attempt = 0
            while attempt < 2:
                try:
                    df = pd.read_parquet(filename, engine='fastparquet')
                    df[df.select_dtypes(include=['number']).columns] = df.select_dtypes(include=['number']).apply(pd.to_numeric, downcast='float')
                    dataframes.append(df)
                    break
                except Exception as e:
                    print(f"Error reading {filename}: {e}")
                    if attempt == 0:
                        os.remove(filename)
                        print("Redownloading...")
                        download_file(full_link, filename, position=1)
                        time.sleep(5)
                    attempt += 1
            
            file_bar.update(1)

    return dataframes

url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
dfs = download_and_read_parquet_files(url)

combined_df = pd.concat(dfs, ignore_index=True)
print(combined_df.head())


Processing files:   0%|          | 0/12 [00:00<?, ?file/s]

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0       2.0  2024-01-01 00:57:55   2024-01-01 01:17:43              1.0   
1       1.0  2024-01-01 00:03:00   2024-01-01 00:09:36              1.0   
2       1.0  2024-01-01 00:17:06   2024-01-01 00:35:01              1.0   
3       1.0  2024-01-01 00:36:38   2024-01-01 00:44:56              1.0   
4       1.0  2024-01-01 00:46:51   2024-01-01 00:52:57              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           1.72         1.0                  N         186.0          79.0   
1           1.80         1.0                  N         140.0         236.0   
2           4.70         1.0                  N         236.0          79.0   
3           1.40         1.0                  N          79.0         211.0   
4           0.80         1.0                  N         211.0         148.0   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \
